In [1]:
from ast import literal_eval
from sklearn.feature_extraction.text import CountVectorizer
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
df = pd.read_csv('/content/drive/My Drive/POS_SCORE.csv')
df.head()

,review_Id,review,score,thumbs_Up_Count,review_App_Version,review_time,label,tokens,pos_tags,scores,compound,modified_scores
0,5854e2fd-b0af-44b5-9458-3efeb2369631,not the smoothest or easiest of apps to naviga...,2,28,2.89.0,2024-02-27 12:49:00,Neutral,"['not', 'smoothest', 'easiest', 'apps', 'navig...","[('not', 'RB'), ('smoothest', 'JJS'), ('easies...","{'neg': 0.066, 'neu': 0.583, 'pos': 0.351, 'co...",0.9643,0.057111
1,2bbc0faa-43a2-4255-8035-23772541cb14,"the app itself has good features, eg let's you...",3,102,2.88.2,2024-10-02 09:35:00,Neutral,"['app', 'ha', 'good', 'feature', 'e', 'Eg', 'l...","[('app', 'JJ'), ('ha', 'NN'), ('good', 'JJ'), ...","{'neg': 0.0, 'neu': 0.794, 'pos': 0.206, 'comp...",0.8858,0.040849
2,204edfd3-a877-4d32-be56-02c1e27abc44,"an easy app to use. easy to navigate, fast at ...",5,84,2.89.0,2024-02-20 11:33:00,Positive,"['easy', 'app', 'use', '.', 'easy', 'navigate'...","[('easy', 'JJ'), ('app', 'NN'), ('use', 'NN'),...","{'neg': 0.078, 'neu': 0.68, 'pos': 0.242, 'com...",0.8799,0.030426
3,1181eb84-aa16-48ea-b455-623e05635048,pointless app. i can see how much i owe and pa...,1,0,2.89.0,2024-02-28 14:06:00,Negative,"['pointless', 'app', '.', 'i', 'can', 'see', '...","[('pointless', 'NN'), ('app', 'NN'), ('.', '.'...","{'neg': 0.144, 'neu': 0.784, 'pos': 0.071, 'co...",-0.5883,-0.013219
4,a074686c-acef-41e8-9e73-c0d9cd5dd04b,update. issue was resolved quickly and i'm rea...,5,2,2.89.0,2024-02-27 16:41:00,Positive,"['update', '.', 'issue', 'WA', 'resolved', 'qu...","[('update', 'NN'), ('.', '.'), ('issue', 'NN')...","{'neg': 0.193, 'neu': 0.542, 'pos': 0.265, 'co...",0.6240,0.034635


In [8]:
df.head()

,review_Id,review,score,thumbs_Up_Count,review_App_Version,review_time,label,tokens,pos_tags
0,5854e2fd-b0af-44b5-9458-3efeb2369631,not the smoothest or easiest of apps to naviga...,2,28,2.89.0,2024-02-27 12:49:00,Neutral,"['not', 'smoothest', 'easiest', 'apps', 'navig...","[('not', 'RB'), ('smoothest', 'JJS'), ('easies..."
1,2bbc0faa-43a2-4255-8035-23772541cb14,"the app itself has good features, eg let's you...",3,102,2.88.2,2024-10-02 09:35:00,Neutral,"['app', 'ha', 'good', 'feature', 'e', 'Eg', 'l...","[('app', 'JJ'), ('ha', 'NN'), ('good', 'JJ'), ..."
2,204edfd3-a877-4d32-be56-02c1e27abc44,"an easy app to use. easy to navigate, fast at ...",5,84,2.89.0,2024-02-20 11:33:00,Positive,"['easy', 'app', 'use', '.', 'easy', 'navigate'...","[('easy', 'JJ'), ('app', 'NN'), ('use', 'NN'),..."
3,1181eb84-aa16-48ea-b455-623e05635048,pointless app. i can see how much i owe and pa...,1,0,2.89.0,2024-02-28 14:06:00,Negative,"['pointless', 'app', '.', 'i', 'can', 'see', '...","[('pointless', 'NN'), ('app', 'NN'), ('.', '.'..."
4,a074686c-acef-41e8-9e73-c0d9cd5dd04b,update. issue was resolved quickly and i'm rea...,5,2,2.89.0,2024-02-27 16:41:00,Positive,"['update', '.', 'issue', 'WA', 'resolved', 'qu...","[('update', 'NN'), ('.', '.'), ('issue', 'NN')..."


In [9]:
# DataFrame df with 'review' as text data and 'label' as corresponding labels
X = df['tokens']
y = df['label']

In [10]:
# Assume 'X' contains the tokenized reviews and 'y' contains the corresponding labels (positive, negative, neutral)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [11]:
tfidf_vectorizer = TfidfVectorizer(max_features=500)

In [12]:
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)

In [13]:
X_test_tfidf = tfidf_vectorizer.transform(X_test)

In [14]:
#Create a Support Vector Machine classifier and Train SVM classifier
svm_classifier = SVC(kernel='linear')
svm_classifier.fit(X_train_tfidf, y_train)

SVC(kernel='linear')

In [15]:
y_pred = svm_classifier.predict(X_test_tfidf)

In [16]:
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print(classification_report(y_test, y_pred))

Accuracy: 0.6666666666666666
              precision    recall  f1-score   support

    Negative       0.68      0.81      0.74        21
     Neutral       0.74      0.56      0.64        25
    Positive       0.56      0.64      0.60        14

    accuracy                           0.67        60
   macro avg       0.66      0.67      0.66        60
weighted avg       0.68      0.67      0.66        60



In [17]:
#LOGISTIC REGRESSION MODEL
from sklearn.linear_model import LogisticRegression

In [18]:
# Create a Logistic Regression classifier and Train the LR classifier
logreg_classifier = LogisticRegression()
# Train the classifier
logreg_classifier.fit(X_train_tfidf, y_train)

LogisticRegression()

In [19]:
y_pred = logreg_classifier.predict(X_test_tfidf)

In [20]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.7166666666666667
Classification Report:
               precision    recall  f1-score   support

    Negative       0.65      0.95      0.77        21
     Neutral       0.83      0.60      0.70        25
    Positive       0.73      0.57      0.64        14

    accuracy                           0.72        60
   macro avg       0.74      0.71      0.70        60
weighted avg       0.74      0.72      0.71        60



In [21]:
#NAIVES BAYES MODEL
from sklearn.naive_bayes import MultinomialNB

In [22]:
# Create a Naives Bayes Model classifier and Train the NB classifier
nb_classifier = MultinomialNB()
nb_classifier.fit(X_train_tfidf, y_train)

MultinomialNB()

In [23]:
y_pred = nb_classifier.predict(X_test_tfidf)

In [24]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.65
Classification Report:
               precision    recall  f1-score   support

    Negative       0.61      0.90      0.73        21
     Neutral       0.64      0.64      0.64        25
    Positive       1.00      0.29      0.44        14

    accuracy                           0.65        60
   macro avg       0.75      0.61      0.61        60
weighted avg       0.71      0.65      0.63        60



In [25]:
#RANDOM FOREST MODEL
from sklearn.ensemble import RandomForestClassifier

In [26]:
# Create a Random Forest classifier and Train the classifier
rf_classifier = RandomForestClassifier(n_estimators=2000, random_state=42)
rf_classifier.fit(X_train_tfidf, y_train)

RandomForestClassifier(n_estimators=2000, random_state=42)

In [27]:
y_pred = rf_classifier.predict(X_test_tfidf)

In [28]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.6333333333333333
Classification Report:
               precision    recall  f1-score   support

    Negative       0.67      0.86      0.75        21
     Neutral       0.62      0.52      0.57        25
    Positive       0.58      0.50      0.54        14

    accuracy                           0.63        60
   macro avg       0.62      0.63      0.62        60
weighted avg       0.63      0.63      0.62        60



In [29]:
#PART OF SPEECH TAGS
# DataFrame df with 'review' as text data and 'label' as corresponding labels
X = df['pos_tags']
y = df['label']

In [30]:
# Assume 'X' contains the pos_tagged tokenized reviews and 'y' contains the corresponding labels (positive, negative, neutral)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [31]:
tfidf_vectorizer = TfidfVectorizer(max_features=500)

In [32]:
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)

In [33]:
X_test_tfidf = tfidf_vectorizer.transform(X_test)

In [34]:
#Create a Support Vector Machine classifier and Train SVM classifier
svm_classifier = SVC(kernel='linear')
svm_classifier.fit(X_train_tfidf, y_train)

SVC(kernel='linear')

In [35]:
y_pred = svm_classifier.predict(X_test_tfidf)

In [36]:
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print(classification_report(y_test, y_pred))

Accuracy: 0.5333333333333333
              precision    recall  f1-score   support

    Negative       0.49      0.95      0.65        21
     Neutral       0.61      0.44      0.51        25
    Positive       1.00      0.07      0.13        14

    accuracy                           0.53        60
   macro avg       0.70      0.49      0.43        60
weighted avg       0.66      0.53      0.47        60



In [37]:
#LOGISTIC REGRESSION MODEL
from sklearn.linear_model import LogisticRegression

In [38]:
# Create a Logistic Regression classifier and Train the LR classifier
logreg_classifier = LogisticRegression()
# Train the classifier
logreg_classifier.fit(X_train_tfidf, y_train)

LogisticRegression()

In [39]:
y_pred = logreg_classifier.predict(X_test_tfidf)

In [40]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.5833333333333334
Classification Report:
               precision    recall  f1-score   support

    Negative       0.53      0.86      0.65        21
     Neutral       0.65      0.52      0.58        25
    Positive       0.67      0.29      0.40        14

    accuracy                           0.58        60
   macro avg       0.62      0.55      0.54        60
weighted avg       0.61      0.58      0.56        60



In [41]:
#NAIVES BAYES MODEL
from sklearn.naive_bayes import MultinomialNB

In [42]:
# Create a Naives Bayes Model classifier and Train the NB classifier
nb_classifier = MultinomialNB()
nb_classifier.fit(X_train_tfidf, y_train)

MultinomialNB()

In [43]:
y_pred = nb_classifier.predict(X_test_tfidf)

In [44]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.48333333333333334
Classification Report:
               precision    recall  f1-score   support

    Negative       0.47      1.00      0.64        21
     Neutral       0.53      0.32      0.40        25
    Positive       0.00      0.00      0.00        14

    accuracy                           0.48        60
   macro avg       0.33      0.44      0.35        60
weighted avg       0.39      0.48      0.39        60



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [45]:
#RANDOM FOREST MODEL
from sklearn.ensemble import RandomForestClassifier

In [46]:
# Create a Random Forest classifier and Train the classifier
rf_classifier = RandomForestClassifier(n_estimators=2000, random_state=42)
rf_classifier.fit(X_train_tfidf, y_train)

RandomForestClassifier(n_estimators=2000, random_state=42)

In [47]:
y_pred = rf_classifier.predict(X_test_tfidf)

In [48]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.6333333333333333
Classification Report:
               precision    recall  f1-score   support

    Negative       0.60      0.86      0.71        21
     Neutral       0.65      0.52      0.58        25
    Positive       0.70      0.50      0.58        14

    accuracy                           0.63        60
   macro avg       0.65      0.63      0.62        60
weighted avg       0.64      0.63      0.62        60

